# Identifying coincidences in Tokyo Olympics 2020 Tweets

***Student:*** Lais Isabelle ALVES DOS SANTOS

### Importing Libraries

In [129]:
import nltk             # Natural Language Toolkit
import pandas as pd
import preprocess as pre
from nltk.sentiment import SentimentIntensityAnalyzer

In [140]:
# if the .py is changed, run to reload
import importlib
importlib.reload(pre)

<module 'preprocess' from 'd:\\Familia\\Lais\\Intercâmbio\\Télécom\\2A\\Creneaux D\\IA225\\mini_project\\ai225-mini-project-coincidences\\preprocess.py'>

### Load data

In [111]:
tokyo_tweets_db = pd.read_csv('../tokyo_2020_tweets.csv')
tokyo_tweets_db

C:\Users\LaisIsabelle\AppData\Local\Temp\ipykernel_14772\941854931.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  tokyo_tweets_db = pd.read_csv('../tokyo_2020_tweets.csv')


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1418888645105356803,Abhishek Srivastav,"Udupi, India",Trying to be mediocre in many things,2021-02-01 06:33:51,45.0,39.0,293.0,False,2021-07-24 10:59:49,Let the party begin\n#Tokyo2020,['Tokyo2020'],Twitter for Android,0.0,0.0,False
1,1418888377680678918,Saikhom Mirabai Channu🇮🇳,"Manipur, India",Indian weightlifter 48 kg category. Champion🏆,2018-04-07 10:10:22,5235.0,5.0,2969.0,False,2021-07-24 10:58:45,Congratulations #Tokyo2020 https://t.co/8OFKMs...,['Tokyo2020'],Twitter for Android,0.0,0.0,False
2,1418888260886073345,Big Breaking,Global,All breaking news related to Financial Market....,2021-05-29 08:51:25,3646.0,3.0,5.0,False,2021-07-24 10:58:17,Big Breaking Now \n\nTokyo Olympic Update \n\n...,NaN,Twitter for Android,0.0,1.0,False
3,1418888172864299008,International Hockey Federation,Lausanne,Official International Hockey Federation Twitt...,2010-10-20 10:45:59,103975.0,2724.0,36554.0,True,2021-07-24 10:57:56,Q4: 🇬🇧3-1🇿🇦\n\nGreat Britain finally find a wa...,NaN,Twitter Web App,1.0,0.0,False
4,1418886894478270464,Cameron Hart,Australia,Football & Tennis Coach,2020-10-31 08:46:17,6.0,37.0,31.0,False,2021-07-24 10:52:51,All I can think of every time I watch the ring...,"['Tokyo2020', 'ArtisticGymnastics', '7Olympics...",Twitter for iPhone,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297006,1419947001102405633,Belgian Cats,Brussels,The Belgian National Team - Women's basketball,2010-05-11 10:28:27,5296,161,973,False,2021-07-27 09:05:21,It's a close contest in our first Olympic game...,NaN,Twitter Web App,7.0,38.0,False
297007,1419946996895346689,IOC MEDIA,Lausanne. Switzerland,Keep updated with all the latest news from the...,2011-01-05 10:00:39,147657,712,624,True,2021-07-27 09:05:20,10 athletes from the IOC @RefugeesOlympic star...,['Tokyo2020'],Twitter Web App,7.0,11.0,False
297008,1419946993909047299,Devin Heroux,"Toronto, Ontario",CBC News/Olympics Reporter. Based in Toronto. ...,2009-03-24 22:53:21,44209,2317,153144,True,2021-07-27 09:05:19,SNACK TIME! \n\nHow could I say no to “two sof...,NaN,Twitter for iPhone,78.0,355.0,False
297009,1419946992948576256,Ⓜ️𝐚𝐬𝐚 🍊 𝐌𝐫.𝐂𝐡𝐢𝐥𝐝𝐫𝐞𝐧 🌏,東京,🗼𝑇𝑜𝑘𝑦𝑜 𝐽𝐴𝑃𝐴𝑁 🇯🇵 ✂︎ 表参道 ⭐︎ 原宿 ✂︎ 𝐻𝑎𝑖𝑟 𝑠𝑡𝑦𝑙𝑖𝑠𝑡 ✂...,2015-12-31 02:13:03,149,245,12208,False,2021-07-27 09:05:19,.\n.\n.\n🏄‍♂️ 𝐊𝐚𝐧𝐨𝐚 𝐈𝐠𝐚𝐫𝐚𝐬𝐡𝐢 🇯🇵\n.\n.\n🥈𝐂𝐨𝐧𝐠𝐫𝐚...,['五十嵐カノア'],Twitter for iPhone,0.0,0.0,False


### Pre-process the data

In [117]:
nltk.download(["stopwords", "vader_lexicon", "punkt"])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LaisIsabelle\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\LaisIsabelle\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LaisIsabelle\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [122]:
# get texts and removes the ones that contains nan
texts = [t for t in tokyo_tweets_db["text"] if str(t) != 'nan']

In [114]:
stopwords = nltk.corpus.stopwords.words("english")      # exclude word "of", "a", "the" and similar to avoid negative effect

In [141]:
sia = SentimentIntensityAnalyzer()
sscores = [sia.polarity_scores(pre.pipeline(text, stopwords)) for text in texts]     # get sentiment scores for each text

In [142]:
# Display some result
id_text = 160
ss = sscores[id_text]

print('Text: {}\nNegative score:\t{}\nNeutral Score:\t{}\nPositive Score:\t{}\nCompound:\t{}'.format(texts[id_text], ss['neg'], ss['neu'], ss['pos'], ss['compound']))

Text: Concentration + satisfaction - Ibrahim Colak 🇹🇷 #Tokyo2020 #ArtisticGymnastics https://t.co/J6itbzNgmI
Negative score:	0.0
Neutral Score:	0.734
Positive Score:	0.266
Compound:	0.4404


### Explore coincidences